In [1]:
import requests,configparser
from IPython.display import display

config = configparser.RawConfigParser()
config.read_file(open(r'../config/stuff.py'))
gc2APIKey = config.get('apiKeys','apiKey_GC2_BCB').replace('"','')
apiBaseURL = "https://gc2.mapcentia.com/api/v1/sql/bcb?key="+gc2APIKey

In [30]:
tirQ = "SELECT s.scientificname_submitted, \
    t.itis \
    FROM public.sgcn s \
    LEFT JOIN tir.tir t \
    ON s.scientificname_submitted = t.registrationinfo -> 'SGCN_ScientificName_Submitted' \
    WHERE t.itis -> 'discoveredTSN' <> 'None' \
    AND s.scientificname_authority IS NULL"
tirR = requests.get(apiBaseURL+"&q="+tirQ).json()
display (tirR)

{'_execution_time': 0.172,
 'auth_check': {'auth_level': 'Write',
  'checked_relations': ['public.sgcn', 'tir.tir'],
  'session': None,
  'success': True},
 'features': [{'properties': {'commonname_authority': 'pink sand verbena',
    'itis': '"Class"=>"Magnoliopsida", "Genus"=>"Abronia", "Order"=>"Caryophyllales", "Family"=>"Nyctaginaceae", "Kingdom"=>"Plantae", "Species"=>"Abronia umbellata", "Variety"=>"Abronia umbellata var. breviflora", "Division"=>"Tracheophyta", "cacheDate"=>"2017-05-01T16:31:56.575985", "Subkingdom"=>"Viridiplantae", "Superorder"=>"Caryophyllanae", "createDate"=>"1996-06-13 14:51:08", "updateDate"=>"2012-06-27 00:00:00", "Subdivision"=>"Spermatophytina", "acceptedTSN"=>"895435", "Infrakingdom"=>"Streptophyta", "Superdivision"=>"Embryophyta", "discoveredTSN"=>"19575", "itisMatchMethod"=>"exact match on Abronia umbellata ssp. breviflora", "vernacular:English"=>"pink sand verbena"',
    'scientificname_authority_species': 'Abronia umbellata',
    'scientificname_s

In [28]:
#for feature in tirR['features']:
#    sourcekey = feature['properties']['sourcekey']
#    scientificname_display = feature['properties']['scientificname']
#    commonname_display = feature['properties']['commonname']
#    if type(scientificname_display) is str:
#        display (scientificname_display)
#        updateQ = "UPDATE public.sgcn SET scientificname_authority = '"+scientificname_display.replace("'","''")+"'"
#        if type(commonname_display) is str:
#            display (commonname_display)
#            updateQ = updateQ+", commonname_authority = '"+commonname_display.replace("'","''")+"'"
#        updateQ = updateQ+"WHERE scientificname_submitted = '"+sourcekey+"'"
#        updateR = requests.get(apiBaseURL+"&q="+updateQ).json()
#        display (updateR)


In [24]:
# Add scientific and common names to the display column when the authority provided them
addAuthorityNamesQ = "UPDATE public.sgcn \
    SET commonname_display = commonname_authority, \
    scientificname_display = scientificname_authority \
    WHERE scientificname_authority IS NOT NULL"
addAuthorityNamesR = requests.get(apiBaseURL+"&q="+addAuthorityNamesQ).json()
display (addAuthorityNamesR)

{'_execution_time': 9.09,
 'affected_rows': 34795,
 'auth_check': {'auth_level': 'Write',
  'checked_relations': ['public.sgcn'],
  'session': None,
  'success': True},
 'success': True}

In [25]:
# Add the submitted common names to the display column when the authority has no common names to provide
addSubmittedNamesQ = "UPDATE public.sgcn \
    SET commonname_display = commonname_submitted \
    WHERE commonname_authority IS NULL \
    AND scientificname_authority IS NOT NULL"
addSubmittedNamesR = requests.get(apiBaseURL+"&q="+addSubmittedNamesQ).json()
display (addSubmittedNamesR)

{'_execution_time': 0.626,
 'affected_rows': 6884,
 'auth_check': {'auth_level': 'Write',
  'checked_relations': ['public.sgcn'],
  'session': None,
  'success': True},
 'success': True}